In [48]:
# !pip4 install datasets
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import AdamW
import tqdm

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# set pad token
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')

dataset = load_dataset('imdb')

Reusing dataset imdb (/Users/taylorsorensen/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
100%|██████████| 3/3 [00:00<00:00, 154.16it/s]


In [78]:
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)

def label_function(examples):
    # replace label with 'positive' if 1 else 'negative'
    return {'target': torch.tensor([1 if examples['label'] == 1 else 0])}

# tokenized_imdb = dataset.map(preprocess_function, batched=True)
tokenized_imdb = dataset.map(label_function, batched=False)
tokenized_imdb['train'][0]

Loading cached processed dataset at /Users/taylorsorensen/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-85f82df3344edf67.arrow
Loading cached processed dataset at /Users/taylorsorensen/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-543c5209b7f4365b.arrow
Loading cached processed dataset at /Users/taylorsorensen/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-9d0e3bd0c89c2429.arrow


{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [49]:
device = 'cpu'
model.train()
model.to(device)
train_dataset = dataset['train']
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

optimizer = AdamW(model.parameters(), lr=0.00001)

In [56]:
pos_token = tokenizer.batch_encode_plus(['positive'], return_tensors='pt').input_ids[0,0]
neg_token = tokenizer.batch_encode_plus(['negative'], return_tensors='pt').input_ids[0,0]
# do one epoch
# for batch in tqdm.tqdm(train_loader):
# single batch
batch = next(iter(train_loader))
# each batch has 'text' and 'label'
optimizer.zero_grad()
# first, tokenize the text (with padding
input_ids = tokenizer.batch_encode_plus(batch['text'], padding=True, return_tensors='pt')['input_ids']
input_ids = input_ids.to(device)
# replace 0 with neg_token and 1 with pos_token
labels = torch.where(batch['label'] == 0, neg_token, pos_token)
labels = labels.to(device)
# print(labels)
# outputs = model(input_ids, labels=labels)
# loss = outputs[0]
# loss.backward()
# optimizer.step()
# output = model(input_ids.T, labels=labels)
# output
input_ids.shape, labels.shape
output = model(input_ids)

In [59]:
output['logits'].shape

torch.Size([4, 550, 50257])

In [63]:
tokenized = tokenizer.batch_encode_plus(batch['text'], padding=True, return_tensors='pt')
tokenized['attention_mask']

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])